In [ ]:
!pip install -U keras
!pip install -U tensorflow

In [ ]:
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=c5b1d2377d502a104c3656003eeec3eddf52548f2a6b9620884e5f4cd7d4c4c9
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
import nltk
import pandas as pd
import re
import numpy as np
import math
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Activation,Conv1D,Embedding,BatchNormalization,GlobalMaxPooling1D,MaxPooling1D,Dropout,Conv2D,Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import to_categorical
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import pad_sequences

# Data processing

In [ ]:
data = 'https://raw.githubusercontent.com/Tong00020/NLP---2024/main/output.csv'

In [ ]:
df_train = pd.read_csv(data, names=["review_text","polarity"])
df_train

<ipython-input-16-26c55ce26321>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(data, names=["review_text","polarity"])


,review_text,polarity
NaN,review_text,polarity
0.0,curtindo,1.0
1.0,aplicativo absurdamente caro valor tornar prêm...,0.0
2.0,simples novo layout,1.0
3.0,estava problema acessar porém consegui resolve...,1.0
...,...,...
1039530.0,legal,1.0
1039531.0,assim ficar dentro situação bolsa família gostei,1.0
1039532.0,NaN,0.0
1039533.0,excelente aplicativo treinar conhecimentos ing...,1.0


In [ ]:
df_train.dropna(inplace = True)

In [ ]:
len(df_train)

901015

In [ ]:
print(f"Training has {df_train.shape} reviews")

Training has (901015, 2) reviews


In [ ]:
corpus = df_train['review_text']

In [ ]:
corpus

NaN                                                review_text
0.0                                                   curtindo
1.0          aplicativo absurdamente caro valor tornar prêm...
2.0                                        simples novo layout
3.0          estava problema acessar porém consegui resolve...
                                   ...                        
1039528.0                                  banido injustamente
1039530.0                                                legal
1039531.0     assim ficar dentro situação bolsa família gostei
1039533.0    excelente aplicativo treinar conhecimentos ing...
1039534.0                                           adoro line
Name: review_text, Length: 901015, dtype: object

In [ ]:
import nltk
nltk.download('punkt')

count_words = {}

for doc in corpus:
    words= nltk.word_tokenize(doc)
    for i in words:
        if i in count_words.keys():
            count_words[i] += 1
        else:
            count_words[i] = 1
count_words

In [ ]:
bag = " ".join(corpus)
tokens = bag.split()
vocab = sorted(set(tokens))
vocab

In [ ]:
import heapq
from sklearn import tree

freq_words = heapq.nlargest(30,count_words, key=count_words.get)
freq_words

['jogo',
 'aplicativo',
 'muito',
 'ótimo',
 'melhor',
 'bom',
 'gostei',
 'legal',
 'consigo',
 'recomendo',
 'excelente',
 'agora',
 'app',
 'fazer',
 'tudo',
 'amei',
 'nada',
 'fica',
 'celular',
 'sempre',
 'conta',
 'atualização',
 'otimo',
 'parabéns',
 'ainda',
 'tempo',
 'usar',
 'todos',
 'ruim',
 'problema']

Inverse Document Frequency (TF-IDF)

In [ ]:
X = corpus
Y = df_train['polarity']

In [ ]:
words_freq = {}
pondered_freq = {}
N = {}

for i,j in enumerate(corpus):
    words_freq[i] = dict()
    pondered_freq[i] = dict()
    words = j.split()
    N[i] = len(words)
    for word in words:
        if word in words_freq[i]:
            words_freq[i][word] += 1
        else:
            words_freq[i][word] = 1

        if word in pondered_freq[i]:
            pondered_freq[i][word] += 1
        else:
            pondered_freq[i][word] = 1

In [ ]:
for i in pondered_freq:
    for word in pondered_freq[i]:
        pondered_freq[i][word] /= N[i]

In [ ]:
len(vocab)

191566

In [ ]:
idf = {}
tf_idf = {}

for word in vocab:
    idf[word] = 0
    for i in words_freq:
        print(i)
        if word in words_freq[i]:
            idf[word] += 1

for word in idf:
      idf[word] = math.log(len(corpus)/idf[word])

for i in pondered_freq:
    tf_idf[i] = dict()
    for word in pondered_freq[i]:
        tf_idf[i][word] = pondered_freq[i][word]*idf[word]

In [ ]:
df = pd.DataFrame().from_records(tf_idf).fillna(0).T
df

In [ ]:
df_term_frequency = df.multiply(idf)
df_term_frequency